# PUPPI Tutorial

This Jupyter notebook demonstrates how to use the `puppi` Python package to process a BioID/AP-MS dataset. We will:

1. Load an example intensity file
2. Run feature engineering
3. Train a PU-learning model and estimate FDR
4. Save the output

In [ ]:
import pandas as pd
from puppi.features import feature_engineering
from puppi.train import train_and_score

## Load Example Data

In [ ]:
input_df = pd.read_csv("input_intensity_dataset.tsv", sep='\t')
input_df.head(10)

## Run Feature Engineering

You must specify control (substrings identifying control samples).

In [ ]:
features_df = feature_engineering(input_df, controls=["EGFP", "Empty", "NminiTurbo"])
features_df.head()

## Train PU-learning Model and Estimate FDR

In [ ]:
final_df = train_and_score(features_df, initial_positives=20, initial_negatives=200)
final_df.head()

## Save Output

In [ ]:
final_df.to_csv("puppi_output.csv", index=False)